# Explore Nextstrain's open data

This notebook assumes you've downloaded nextstrain's [open metadata](https://docs.nextstrain.org/projects/ncov/en/latest/reference/remote_inputs.html):
```sh
mkdir -p results/nextstrain
gsutil cp gs://nextstrain-data/files/ncov/metadata.tsv.gz results/nextstrain/
python scripts/preprocess_nextstrain.py
```

In [1]:
import torch
import pickle
import matplotlib.pyplot as plt

In [2]:
columns = pickle.load(open("results/nextstrain.columns.pkl", "rb"))
stats = pickle.load(open("results/nextstrain.stats.pkl", "rb"))
features = torch.load("results/nextstrain.features.pt")

In [3]:
print(len(columns["lineage"]))

74861
